In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem import Descriptors
# from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

In [2]:
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [3]:
train_df = pd.read_csv('../input/train.csv')
train_df['AlogP'] = np.where(pd.isna(train_df['AlogP']), train_df['LogD'], train_df['AlogP'])
test_df = pd.read_csv('../input/test.csv')
test_df['AlogP'] = np.where(pd.isna(test_df['AlogP']), test_df['LogD'], test_df['LogD'])

In [4]:
train_df = train_df.drop_duplicates('SMILES', keep=False).reset_index(drop=True)

In [5]:
target_hlm = train_df['HLM']
target_mlm = train_df['MLM']

In [6]:
train_df['Molecule'] = train_df['SMILES'].apply(Chem.MolFromSmiles)
test_df['Molecule'] = test_df['SMILES'].apply(Chem.MolFromSmiles)
train_desc = [getMolDescriptors(m) for m in train_df['Molecule']]
test_desc = [getMolDescriptors(m) for m in test_df['Molecule']]

In [7]:
train_desc = pd.DataFrame(train_desc)
test_desc = pd.DataFrame(test_desc)

In [8]:
null_list = []
for col in train_desc.columns:
  missing = train_desc[col].isna().any()
  if missing == True:
    null_list.append(col)

In [9]:
train_desc = train_desc.drop(null_list, axis=1)
test_desc = test_desc.drop(null_list, axis=1)

In [10]:
train = pd.concat([train_df, train_desc],axis=1).drop(columns=['id','SMILES','Molecule','MLM','HLM','MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])
test = pd.concat([test_df, test_desc],axis=1).drop(columns=['id','SMILES','Molecule','MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

In [11]:
scaler = MinMaxScaler()

In [12]:
features = [col for col in train.columns]
train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])

In [13]:
import dgl
import dgl.function as fn

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
ATOM_VOCAB = [
	'C', 'N', 'O', 'S', 'F',
	'H', 'Si', 'P', 'Cl', 'Br',
	'Li', 'Na', 'K', 'Mg', 'Ca',
	'Fe', 'As', 'Al', 'I', 'B',
	'V', 'Tl', 'Sb', 'Sn', 'Ag',
	'Pd', 'Co', 'Se', 'Ti', 'Zn',
	'Ge', 'Cu', 'Au', 'Ni', 'Cd',
	'Mn', 'Cr', 'Pt', 'Hg', 'Pb'
]
def one_of_k_encoding(x, vocab):
  if x not in vocab:
    print(f"Not in ATOM_VOCAB: {x}")
    x = vocab[-1]
  return list(map(lambda s: float(x==s), vocab))


def get_atom_feature(atom):
	atom_feature = one_of_k_encoding(atom.GetSymbol(), ATOM_VOCAB)   #c면 [1,0,0,0,....] n이면[0,1,0,0,0,.....]
	atom_feature += one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) #다리가 몇갠지
	atom_feature += one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4])
	atom_feature += one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
	atom_feature += [atom.GetIsAromatic()]
	return atom_feature

def get_bond_feature(bond):
  bt = bond.GetBondType()
  bond_feature = [
      bt == Chem.rdchem.BondType.SINGLE,
      bt == Chem.rdchem.BondType.DOUBLE,
      bt == Chem.rdchem.BondType.TRIPLE,
      bt == Chem.rdchem.BondType.AROMATIC,
      bond.GetIsConjugated(),  #결합체인가
      bond.IsInRing()           #링안에 있는지
  ]
  return bond_feature

In [16]:
def get_molecular_graph(smi):
  mol = Chem.MolFromSmiles(smi)
  graph = dgl.DGLGraph()

  atom_list = mol.GetAtoms()
  num_atoms = len(atom_list)
  graph.add_nodes(num_atoms)

  atom_feature_list = [get_atom_feature(atom) for atom in atom_list]
  atom_feature_list = torch.tensor(atom_feature_list, dtype=torch.float64)
  graph.ndata['h'] = atom_feature_list

  bond_list = mol.GetBonds()
  bond_feature_list = []
  for bond in bond_list:
    bond_feature = get_bond_feature(bond)

    src = bond.GetBeginAtom().GetIdx() #엣지 시작점
    dst = bond.GetEndAtom().GetIdx()   #끝점

    ## DGL 그래프는 방향성이 없어
    ## 쌍으로 줘야함
    # i --> j
    graph.add_edges(src, dst)
    bond_feature_list.append(bond_feature)
    # j --> i
    graph.add_edges(dst, src)
    bond_feature_list.append(bond_feature)

  bond_feature_list = torch.tensor(bond_feature_list, dtype = torch.float64)
  graph.edata['e_ij'] = bond_feature_list
  return graph

In [17]:
graph_list = []
for smi in train_df['SMILES']:
  graph = get_molecular_graph(smi)
  graph_list.append(graph)

TypeError: graph() missing 1 required positional argument: 'data'

In [ ]:
graph_list[0]

In [ ]:
def my_collate_fn(batch):
  graph_list=[]
  for item in batch:
    graph = get_molecular_graph(item[1])
    graph_list.append(graph)
  graph_list = dgl.batch(graph_list)
  features = item[0]
  label = item[2]
  return torch.tensor(features).float(), graph_list, label

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MolDataset(Dataset):
  def __init__(self, df, graph, label=None, test=False):
    self.df = df
    self.graph = graph
    self.label = label
    self.test = test
  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    if not self.test:
      return self.df.iloc[idx].values, self.graph[idx], torch.tensor(self.label[idx]).float()
    else:
      return self.df.iloc[idx].values, self.graph[idx]

In [ ]:
hlm_dataset = MolDataset(df = train, graph = train_df['SMILES'], label = target_hlm, test= False)

In [ ]:
input_size = hlm_dataset.df.shape[1]  #199
graph_nsize = 58#graph.ndata['h'].shape[1]   #58
graph_esize = 6#graph.edata['e_ij'].shape[1]   #6

In [ ]:
train_hlm_data, valid_hlm_data = train_test_split(hlm_dataset, test_size=0.2, random_state=42)

In [ ]:
train_hlm_loader = DataLoader(dataset=train_hlm_data, batch_size=256, shuffle=True, collate_fn = my_collate_fn)
val_hlm_loader = DataLoader(dataset=valid_hlm_data, batch_size=256, shuffle=False, collate_fn = my_collate_fn)

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.linear1 = nn.Linear(58, 256)
    self.linear2 = nn.Linear(256, 128)
  def forward(self, h):
    h = self.linear1(h)
    h = F.relu(h)
    h = self.linear2(h)
    return h

In [ ]:
class GraphConvolution(nn.Module):
  def __init__(self):
    super().__init__()
    self.norm = nn.LayerNorm(128)
    self.linear = nn.Linear(128,128,bias=False)

  def forward(self, graph, training=False):
    h0 = graph.ndata['h']

    graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'u_'))
    h = F.relu(self.linear(graph.ndata['u_'])) + h0
    h = self.norm(h)

    h = F.dropout(h, p=0.2, training = training)

    graph.ndata['h'] = h
    return graph



In [ ]:
class MyModel(nn.Module):
  def __init__(self,num_layers=4, graph_nsize=58, graph_esize=6, readout='sum', input_size=199):
    super().__init__()
    ####
    self.num_layers = num_layers
    self.embedding_node = nn.Linear(graph_nsize, 128, bias=False)
    self.embedding_edge = nn.Linear(graph_esize, 128, bias=False)
    self.readout = readout

    self.mp_layers = nn.ModuleList()
    for _ in range(self.num_layers):
      mp_layer = None
      mp_layer = GraphConvolution()
      self.mp_layers.append(mp_layer)

    self.linear_out = nn.Linear(128, 1, bias=False)
    #####
    self.fc1 = nn.Linear(input_size, 256)
    self.bn1 = nn.LayerNorm(256)
    self.dropout1 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(256, 512)
    self.bn2 = nn.LayerNorm(512)
    self.dropout2 = nn.Dropout(0.2)

    self.fc3 = nn.Linear(1, 1024)
    self.bn3 = nn.LayerNorm(1024)
    self.dropout3 = nn.Dropout(0.2)
    self.fc4 = nn.Linear(1024, 512)
    self.bn4 = nn.LayerNorm(512)
    self.dropout4 = nn.Dropout(0.2)
    self.fc5 = nn.Linear(512, 256)
    self.bn5 = nn.LayerNorm(256)
    self.dropout5 = nn.Dropout(0.2)
    self.fc6 = nn.Linear(256, 128)
    self.bn6 = nn.LayerNorm(128)
    self.dropout6 = nn.Dropout(0.2)
    self.fc7 = nn.Linear(128, 1)
  def forward(self, graph, feature, training=False):
    h = self.embedding_node(graph.ndata['h'].float())
    e_ij = self.embedding_edge(graph.edata['e_ij'].float())
    graph.ndata['h'] = h
    graph.edata['e_ij'] = e_ij

    for i in range(self.num_layers):
      graph = self.mp_layers[i](graph)

    x1 = dgl.readout_nodes(graph, 'h', op=self.readout)
    x1 = self.linear_out(x1)

    x2 = self.fc1(feature)
    x2 = self.bn1(x2)
    x2 = torch.relu(x2)
    x2 = self.dropout1(x2)
    x2 = torch.relu(self.bn2(self.fc2(x2)))
    x2 = self.dropout2(x2)
    x2 = x2.view(-1,1)

    x = torch.cat((x1,x2), dim=0)
    x = torch.relu(self.bn3(self.fc3(x)))
    x = self.dropout3(x)
    x = torch.relu(self.bn4(self.fc4(x)))
    x = self.dropout4(x)
    x = torch.relu(self.bn5(self.fc5(x)))
    x = self.dropout5(x)
    x = torch.relu(self.bn6(self.fc6(x)))
    x = self.dropout6(x)
    out = self.fc7(x)
    return(out)

In [ ]:
model = MyModel(num_layers=4, graph_nsize=graph_nsize, graph_esize=graph_esize, readout='sum',input_size=input_size)

In [ ]:
import torch.optim as optim
import math

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_hat, y):
        loss = torch.sqrt(self.mse(y_hat,y))
        return loss

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = RMSELoss()

In [ ]:
num_epochs = 50
losses = []
for epoch in range(num_epochs):
  model.train()
  running_loss = 0
  for feat, grap, label in train_hlm_loader:
    optimizer.zero_grad()
    outputs = model(graph=grap, feature=feat, training=True)

    loss = criterion(outputs, label.view(-1, 1))
    loss.backward()
    running_loss += loss.item()
    optimizer.step()

  print(running_loss)

In [ ]:
math.sqrt(running_loss)

In [ ]:
np.mean(losses)

In [ ]:
a.ndata['h'].shape

In [ ]:
a.edata['e_ij'].shape

In [ ]:
embedding_node = nn.Linear(58, 128)
embedding_edge = nn.Linear(6, 128)

In [ ]:
n = a.ndata['h'].float()
n = embedding_node(n)
e = a.edata['e_ij'].float()
e = embedding_edge(e)

In [ ]:
a.ndata['h'] = n
a.edata['e_ij'] = e

In [ ]:
h0 = a.ndata['h']
a.update_all(fn.copy_u('h', 'm'), fn.sum('m','u_'))
a

In [ ]:
lin = nn.Linear(128, 128)
nrom = nn.LayerNorm(128)

In [ ]:
h = F.relu(lin(a.ndata['u_'])) + h0

In [ ]:
a.ndata['h'] = h

In [ ]:
h = F.dropout(h, p=0.2)

In [ ]:
dgl.readout_nodes(a, 'h', op='sum').shape

In [ ]:
h.shape

In [ ]:
d = torch.randn(4)

In [ ]:
d.view(-1,1)